In [2]:
import math
import datetime
import os, sys
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from IPython.display import Audio
import csv

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torchsummary import summary

import cv2




# local files
sys.path.insert(0, '../')
import pyClient
import utils
import model
from model import Transition

In [3]:
# Training parameters
BATCH_SIZE = 128 #original 128
GAMMA = 0.999
EPS_START = 0.95
EPS_END = 0.05
EPS_DECAY_steps = 4000 
EPS_DECAY = (EPS_START - EPS_END)/EPS_DECAY_steps
REPLAY_START_SIZE =  128 #TODO PUT BACK TO 1500 #steps taken
TARGET_UPDATE = 10 #episodes
DEVICE = 'cuda:0'

# Environment parameters
IMSIZE = 128
STACK_SIZE = 1
N_ACTIONS = 3
IP  = "127.0.0.1" # Ip address that the TCP/IP interface listens to
PORT = 13000       # Port number that the TCP/IP interface listens to


environment = pyClient.Environment(ip = IP, port = PORT, size = IMSIZE) 
agent = model.DoubleDQNAgent(imsize=IMSIZE,
                 in_channels=STACK_SIZE,
                 n_actions=N_ACTIONS,
                 memory_capacity=12000,
                 eps_start=EPS_START,
                 eps_end=EPS_END,
                 eps_delta=EPS_DECAY,
                 gamma_discount = GAMMA,
                 batch_size = BATCH_SIZE,
                 device=DEVICE)



In [4]:
# Utilities
def process_state(state_raw):
    """ @TODO """
    frame = environment.state2usableArray(state_raw)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = frame.astype('float32')
    return torch.Tensor(frame / 255.).view(1,1,environment.size, environment.size)

In [6]:
MAX_EPISODES = 1e5
MAX_STEPS  = 1e7
TRAINING_CONDITION = 0
LOGFILE = 'Experiments/Out/test-01.csv'

RESET_UPON_END_SIGNAL = {0:False,  # Nothing happened
                         1:True,   # Box collision
                         2:True,   # Wall collision
                         3:False}  # Reached step target



with open(LOGFILE, 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['episode','step_count', 'train_loss', 'reward'])

lr_dqn = 0.01
optimizer = optim.Adam(agent.policy_net.parameters(), lr = lr_dqn)
total_reward = 0
total_loss = 0
for episode in range(int(MAX_EPISODES)):
    with open(LOGFILE, 'a') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        writer.writerow([episode,agent.step_count,total_reward,total_loss]) 
    
    
    if agent.step_count > MAX_STEPS:
        break
    
    if episode % TARGET_UPDATE == 0:  #episodes
        print('episode {}, target net updated'.format(episode))
        agent.update_target_net
    
    _, _, state_raw = environment.reset(TRAINING_CONDITION)
    state = process_state(state_raw).to(DEVICE)

    for t in count(): 
        action = agent.select_action(state)
        end, reward, next_state_raw = environment.step(action.item())
        next_state = process_state(next_state_raw).to(DEVICE) if not RESET_UPON_END_SIGNAL[end] else None
        if reward > 100:
            reward = -(reward -100)
        reward = torch.tensor([reward], device=DEVICE,dtype=torch.float)
        total_reward += reward.item();
        action = action.unsqueeze(0)
        agent.memory.push(state, action, next_state, reward)

        if RESET_UPON_END_SIGNAL[end]:
            break
            
        if len(agent.memory) > REPLAY_START_SIZE:
            
            state_action_values, expected_state_action_values = agent.forward()
            
            """ @TODO: Optimize model"""
            # Compute Huber loss
            loss = F.smooth_l1_loss(state_action_values, expected_state_action_values)
            total_loss += loss.item()

            # Optimize the model
            optimizer.zero_grad()
            loss.backward()

            nn.utils.clip_grad_norm_(agent.policy_net.parameters(), 1)

            optimizer.step()
        else:
            agent.step_count = 0

episode 0, target net updated
episode 10, target net updated
episode 20, target net updated
episode 30, target net updated
episode 40, target net updated
episode 50, target net updated
episode 60, target net updated
episode 70, target net updated
episode 80, target net updated
episode 90, target net updated
episode 100, target net updated
episode 110, target net updated
episode 120, target net updated
episode 130, target net updated
episode 140, target net updated
episode 150, target net updated
episode 160, target net updated
episode 170, target net updated
episode 180, target net updated
episode 190, target net updated
episode 200, target net updated
episode 210, target net updated
episode 220, target net updated
episode 230, target net updated
episode 240, target net updated
episode 250, target net updated
episode 260, target net updated
episode 270, target net updated
episode 280, target net updated
episode 290, target net updated
episode 300, target net updated
episode 310, target

episode 2520, target net updated
episode 2530, target net updated
episode 2540, target net updated
episode 2550, target net updated
episode 2560, target net updated
episode 2570, target net updated
episode 2580, target net updated
episode 2590, target net updated
episode 2600, target net updated
episode 2610, target net updated
episode 2620, target net updated
episode 2630, target net updated
episode 2640, target net updated
episode 2650, target net updated
episode 2660, target net updated
episode 2670, target net updated
episode 2680, target net updated
episode 2690, target net updated
episode 2700, target net updated
episode 2710, target net updated
episode 2720, target net updated
episode 2730, target net updated
episode 2740, target net updated
episode 2750, target net updated
episode 2760, target net updated
episode 2770, target net updated
episode 2780, target net updated
episode 2790, target net updated
episode 2800, target net updated
episode 2810, target net updated
episode 28

episode 5010, target net updated
episode 5020, target net updated
episode 5030, target net updated
episode 5040, target net updated
episode 5050, target net updated
episode 5060, target net updated
episode 5070, target net updated
episode 5080, target net updated
episode 5090, target net updated
episode 5100, target net updated
episode 5110, target net updated
episode 5120, target net updated
episode 5130, target net updated
episode 5140, target net updated
episode 5150, target net updated
episode 5160, target net updated
episode 5170, target net updated
episode 5180, target net updated
episode 5190, target net updated
episode 5200, target net updated
episode 5210, target net updated
episode 5220, target net updated
episode 5230, target net updated
episode 5240, target net updated
episode 5250, target net updated
episode 5260, target net updated
episode 5270, target net updated
episode 5280, target net updated
episode 5290, target net updated
episode 5300, target net updated
episode 53

episode 7500, target net updated
episode 7510, target net updated
episode 7520, target net updated
episode 7530, target net updated
episode 7540, target net updated
episode 7550, target net updated
episode 7560, target net updated
episode 7570, target net updated
episode 7580, target net updated
episode 7590, target net updated
episode 7600, target net updated
episode 7610, target net updated
episode 7620, target net updated
episode 7630, target net updated
episode 7640, target net updated
episode 7650, target net updated
episode 7660, target net updated
episode 7670, target net updated
episode 7680, target net updated
episode 7690, target net updated
episode 7700, target net updated
episode 7710, target net updated
episode 7720, target net updated
episode 7730, target net updated
episode 7740, target net updated
episode 7750, target net updated
episode 7760, target net updated
episode 7770, target net updated
episode 7780, target net updated
episode 7790, target net updated
episode 78

episode 9990, target net updated
episode 10000, target net updated
episode 10010, target net updated
episode 10020, target net updated
episode 10030, target net updated
episode 10040, target net updated
episode 10050, target net updated
episode 10060, target net updated
episode 10070, target net updated
episode 10080, target net updated
episode 10090, target net updated
episode 10100, target net updated
episode 10110, target net updated
episode 10120, target net updated
episode 10130, target net updated
episode 10140, target net updated
episode 10150, target net updated
episode 10160, target net updated
episode 10170, target net updated
episode 10180, target net updated
episode 10190, target net updated
episode 10200, target net updated
episode 10210, target net updated
episode 10220, target net updated
episode 10230, target net updated
episode 10240, target net updated
episode 10250, target net updated
episode 10260, target net updated
episode 10270, target net updated
episode 10280, 

...

episode 50960, target net updated
episode 50970, target net updated
episode 50980, target net updated
episode 50990, target net updated
episode 51000, target net updated
episode 51010, target net updated
episode 51020, target net updated
episode 51030, target net updated
episode 51040, target net updated
episode 51050, target net updated
episode 51060, target net updated
episode 51070, target net updated
episode 51080, target net updated
episode 51090, target net updated
episode 51100, target net updated
episode 51110, target net updated
episode 51120, target net updated
episode 51130, target net updated
episode 51140, target net updated
episode 51150, target net updated
episode 51160, target net updated
episode 51170, target net updated
episode 51180, target net updated
episode 51190, target net updated
episode 51200, target net updated
episode 51210, target net updated
episode 51220, target net updated
episode 51230, target net updated
episode 51240, target net updated
episode 51250,

episode 53370, target net updated
episode 53380, target net updated
episode 53390, target net updated
episode 53400, target net updated
episode 53410, target net updated
episode 53420, target net updated
episode 53430, target net updated
episode 53440, target net updated
episode 53450, target net updated
episode 53460, target net updated
episode 53470, target net updated
episode 53480, target net updated
episode 53490, target net updated
episode 53500, target net updated
episode 53510, target net updated
episode 53520, target net updated
episode 53530, target net updated
episode 53540, target net updated
episode 53550, target net updated
episode 53560, target net updated
episode 53570, target net updated
episode 53580, target net updated
episode 53590, target net updated
episode 53600, target net updated
episode 53610, target net updated
episode 53620, target net updated
episode 53630, target net updated
episode 53640, target net updated
episode 53650, target net updated
episode 53660,

episode 55780, target net updated
episode 55790, target net updated
episode 55800, target net updated
episode 55810, target net updated
episode 55820, target net updated
episode 55830, target net updated
episode 55840, target net updated
episode 55850, target net updated
episode 55860, target net updated
episode 55870, target net updated
episode 55880, target net updated
episode 55890, target net updated
episode 55900, target net updated
episode 55910, target net updated
episode 55920, target net updated
episode 55930, target net updated
episode 55940, target net updated
episode 55950, target net updated
episode 55960, target net updated
episode 55970, target net updated
episode 55980, target net updated
episode 55990, target net updated
episode 56000, target net updated
episode 56010, target net updated
episode 56020, target net updated
episode 56030, target net updated
episode 56040, target net updated
episode 56050, target net updated
episode 56060, target net updated
episode 56070,

episode 58190, target net updated
episode 58200, target net updated
episode 58210, target net updated
episode 58220, target net updated
episode 58230, target net updated
episode 58240, target net updated
episode 58250, target net updated
episode 58260, target net updated
episode 58270, target net updated
episode 58280, target net updated
episode 58290, target net updated
episode 58300, target net updated
episode 58310, target net updated
episode 58320, target net updated
episode 58330, target net updated
episode 58340, target net updated
episode 58350, target net updated
episode 58360, target net updated
episode 58370, target net updated
episode 58380, target net updated
episode 58390, target net updated
episode 58400, target net updated
episode 58410, target net updated
episode 58420, target net updated
episode 58430, target net updated
episode 58440, target net updated
episode 58450, target net updated
episode 58460, target net updated
episode 58470, target net updated
episode 58480,

KeyboardInterrupt: 